In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import obspy
from obspy.core import read, UTCDateTime
from tqdm import tqdm
import seisbench
import seisbench.models as sbm
import glob
import os

## Settings directory

In [2]:
# parent directory untuk save file hasil picking
output_dir = "D:/SERD/01_meq/asosiasi/region/rantau_dedap/" # ganti**************************

# input waveform (parent directory)
input_dir = "D:/SERD/01_meq/2024/2024 12/"

## Functions

In [3]:
# Functions
def cari_folder(path):
    data_folder_path = path
    file_paths = glob.glob(os.path.join(data_folder_path, '*'))
    file_names = [os.path.basename(f) for f in file_paths]
    data_df = pd.DataFrame(file_names, columns=['fname'])

    return data_df

In [4]:
list_julianday = cari_folder(input_dir)
list_julianday

,fname
0,20241201_336
1,20241202_337
2,20241203_338
3,20241204_339
4,20241205_340
5,20241206_341
6,20241207_342
7,20241208_343
8,20241209_344
9,20241210_345


## Model deep learning

In [5]:
# import model deep learning
pn = sbm.PhaseNet.from_pretrained("original") # phasenet original weight
volpick = sbm.PhaseNet.from_pretrained("volpick") # phasenet yang telah dilatih dengan data gempa VT dan LP
eqt = sbm.EQTransformer.from_pretrained("original") # eqtransformer original weight

## Main code

In [6]:
pick_df = []

for i in tqdm(range(len(list_julianday))):
    list_hours = cari_folder(input_dir+list_julianday.fname[i])
    
    for j in range(len(list_hours)):
        list_waveform = cari_folder(input_dir+list_julianday.fname[i]+"/"+list_hours.fname[j]+"/")
        list_waveform['fname'] = list_waveform['fname'].str.replace(r'_(BHE|BHN|BHZ)', '', regex=True).str.replace('.mseed','')
        filename = list_waveform.fname.unique()
        
        for k in range(len(filename)):  
            stream = obspy.read(input_dir+list_julianday.fname[i]+"/"+list_hours.fname[j]+"/"+filename[k]+"*")
            picks = pn.classify(stream, P_threshold=0.1, S_threshold=0.1).picks

            for p in picks:
                pick_df.append({
                    "id": p.trace_id,
                    "timestamp": p.peak_time.datetime,
                    "prob": p.peak_value,
                    "type": p.phase.upper(),
                    'filename':filename[k]
                })

pick_df = pd.DataFrame(pick_df)
pick_df['id'] = pick_df['id'].str.replace(r'^RD\.', '', regex=True).str.replace(r'\.', '', regex=True)
pick_df

  3%|██▌                                                                             | 1/31 [02:16<1:08:14, 136.48s/it]2025-01-26 10:59:36,049 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples or misaligned traces. Output might be empty.
2025-01-26 10:59:36,049 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
100%|███████████████████████████████████████████████████████████████████████████████| 31/31 [1:09:56<00:00, 135.37s/it]


,id,timestamp,prob,type,filename
0,RD01,2024-12-01 00:01:55.540,0.129224,P,RD01_(B144)_RD_20241201_000000
1,RD01,2024-12-01 00:01:55.600,0.140935,S,RD01_(B144)_RD_20241201_000000
2,RD01,2024-12-01 00:08:04.780,0.154790,S,RD01_(B144)_RD_20241201_000000
3,RD01,2024-12-01 00:10:59.030,0.126697,P,RD01_(B144)_RD_20241201_000000
4,RD01,2024-12-01 00:11:07.040,0.357689,S,RD01_(B144)_RD_20241201_000000
...,...,...,...,...,...
362213,RD15,2024-12-31 23:46:42.150,0.135181,P,RD15_(AD18)_RD_20241231_230000
362214,RD15,2024-12-31 23:52:07.990,0.143809,P,RD15_(AD18)_RD_20241231_230000
362215,RD15,2024-12-31 23:52:10.440,0.122851,P,RD15_(AD18)_RD_20241231_230000
362216,RD15,2024-12-31 23:52:10.440,0.152755,S,RD15_(AD18)_RD_20241231_230000


In [7]:
pick_df.to_csv(output_dir+"pn_picks202412.csv",index=False)